# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from scipy.optimize import root_scalar

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

import pandas as pd
import math
import datetime as dt
import pytz
from pandas import DataFrame
import numpy  as np

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [5]:
#Obtención de datos
Datos=pd.read_excel('../data/SOFR-10012019-10292020.xls','nice format')
Datos=Datos.iloc[::-1] #orden e índice
Datos=Datos.reset_index(drop=True)


In [6]:
#Formato
for i in range(np.size(Datos,0)):
    Datos.iloc[i,0]=dt.datetime.strptime(Datos.iloc[i,0],'%Y-%m-%d')
Datos.style.format({'DATE': '{%Y-%m-%d}'})
Datos.style.format({'RATE\n(PERCENT)': '{0:.2%}'})


for i in range(np.size(Datos,0)):
     Datos.iloc[i,0] = Qcf.QCDate(Datos.iloc[i,0].day, Datos.iloc[i,0].month, Datos.iloc[i,0].year)#se pasa las fechas a formato qcdate
        
Datos['RATE\n(PERCENT)']=Datos['RATE\n(PERCENT)']/100 #se obtienen las tasas en no porcentaje


In [7]:
Datos #Visualizar

,DATE,BENCHMARK NAME,RATE\n(PERCENT)
0,1-10-2019,SOFR,0.0188
1,2-10-2019,SOFR,0.0185
2,3-10-2019,SOFR,0.0184
3,4-10-2019,SOFR,0.0182
4,7-10-2019,SOFR,0.0183
...,...,...,...
265,22-10-2020,SOFR,0.0007
266,23-10-2020,SOFR,0.0008
267,26-10-2020,SOFR,0.0009
268,27-10-2020,SOFR,0.0009


In [8]:
Datos=Datos[['DATE','RATE\n(PERCENT)']] #Se toman las columnas pertinentes
Datos.columns=['Fecha','Tasa']
Datos

,Fecha,Tasa
0,1-10-2019,0.0188
1,2-10-2019,0.0185
2,3-10-2019,0.0184
3,4-10-2019,0.0182
4,7-10-2019,0.0183
...,...,...
265,22-10-2020,0.0007
266,23-10-2020,0.0008
267,26-10-2020,0.0009
268,27-10-2020,0.0009


In [9]:
qc_tasa=[]
for i in range(np.size(Datos,0)):
    qc_tasa.append(Qcf.QCInterestRate(Datos.iloc[i,1], Qcf.QCAct360(), Qcf.QCLinearWf()))#Tasa overnight en formato qc
Datos['Tasa Overnight']=qc_tasa
Datos

,Fecha,Tasa,Tasa Overnight
0,1-10-2019,0.0188,0.018800 Act360 Lin
1,2-10-2019,0.0185,0.018500 Act360 Lin
2,3-10-2019,0.0184,0.018400 Act360 Lin
3,4-10-2019,0.0182,0.018200 Act360 Lin
4,7-10-2019,0.0183,0.018300 Act360 Lin
...,...,...,...
265,22-10-2020,0.0007,0.000700 Act360 Lin
266,23-10-2020,0.0008,0.000800 Act360 Lin
267,26-10-2020,0.0009,0.000900 Act360 Lin
268,27-10-2020,0.0009,0.000900 Act360 Lin


In [10]:
FCapital=[]
dia_final=Qcf.QCDate(29, 10, 2020)#Fecha siguiente al último dia
for i in range(np.size(Datos,0)-1):
          FCapital.append(qc_tasa[i].wf(Datos.iloc[i,0], Datos.iloc[i+1,0]))#Calculo del WF
FCapital.append(qc_tasa[269].wf(Datos.iloc[269,0], dia_final))#Ultimo dia requiere una fecha extra, se agrega al final
Datos['WF']=FCapital
Datos

,Fecha,Tasa,Tasa Overnight,WF
0,1-10-2019,0.0188,0.018800 Act360 Lin,1.000052
1,2-10-2019,0.0185,0.018500 Act360 Lin,1.000051
2,3-10-2019,0.0184,0.018400 Act360 Lin,1.000051
3,4-10-2019,0.0182,0.018200 Act360 Lin,1.000152
4,7-10-2019,0.0183,0.018300 Act360 Lin,1.000051
...,...,...,...,...
265,22-10-2020,0.0007,0.000700 Act360 Lin,1.000002
266,23-10-2020,0.0008,0.000800 Act360 Lin,1.000007
267,26-10-2020,0.0009,0.000900 Act360 Lin,1.000002
268,27-10-2020,0.0009,0.000900 Act360 Lin,1.000002


In [11]:
fixings=Qcf.time_series()   #Se inicializa el time_series
for i in range(np.size(Datos,0)):
    fixings[Datos.iloc[i,0]]=Datos.iloc[i,1] #Se guerda el WF para la fecha i en el time series
    

In [12]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        WF=[]
        fechas=Qcf.time_series_dates(fixings) #Obtener fechas del fixings
        tasas=[]
        j=0
        for i in fechas: #Recorrer las fechas
            if i>=self.start_date: #Si llegamos al start date
                z=j #Guardar el indice del start.date
                tasas.append(Qcf.QCInterestRate(fixings[i], Qcf.QCAct360(), Qcf.QCLinearWf())) #Calcular la tasa
                if i>=accrual_date: #Si terminamos el periodo a calcular, salir del ciclo
                    break
            j+=1 #Recorrer indices
                    
        for i in range(np.size(tasas,0)):
                WF.append(tasas[i].wf(fechas[z],fechas[z+1])) #Calculo del WF para las fechas pertinentes
                
            
        P=np.product(WF) #Obtener el productos de WF
        dias = self.start_date.day_diff(accrual_date) #diferencia de dias
        
        tasa_eqv=(P-1)*360/dias #Calculo de tasa equivalente
        
        return tasa_eqv
                

    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        tasa_eqv=self.get_accrued_rate(accrual_date,fixings) #llamar a tasa equivalente 
        dias = self.start_date.day_diff(accrual_date) #Diferencia de dias
        interest=self.notional*tasa_eqv*dias/360 #Obtener iteres del nocional
        
        return interest
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        # amount=self.amortization+self.get_accrued_intereset(self.end_date,fixings) #Amortización+Intereses
        # Corregido por AD
        amount=self.amortization+self.get_accrued_interest(self.end_date,fixings) #Amortización+Intereses
        
        return amount
    
    def test(self):
        return 'test'
    
    def test1(self, x: float):
        if x == 0:
            raise ValueError('x debe ser != 0')
        return 1 / x
    
    def test2(self, x: float):
        return self.test1(x) + 1
    
    def test3(self, x: float):
        return x + self.notional

Construcción de un objeto `Qcf.InterestRateIndex`.

In [13]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [14]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(28, 10, 2020),
    Qcf.QCDate(28, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [15]:
try:
    ois.test1(0)
except Exception as e:
    print('Hacer algo')

Hacer algo


In [16]:
print(ois.start_date)

1-10-2019


In [17]:
print(ois.notional)

10000000


In [18]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx') #Curva zero cupón
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

val_date=Qcf.QCDate(10, 10, 2019)

## Funciones

In [19]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    
    
    if val_date<ois_cashflow.start_date: #En caso de que el val_date fuera antes de que las fechas conocidas

        p1 = val_date.day_diff(ois_cashflow.start_date) #Se usa la Curva Zero Cupón para todo el períoo
        p2 = val_date.day_diff(ois_cashflow.end_date)


        df1 = zcc.get_discount_factor_at(p1)
        df2 = zcc.get_discount_factor_at(p2)
        df_largo=df2 #Guardar el factor de descuento para el present value
        el = (df1 / df2 - 1) * 360.0 / (p2 - p1)

        ois_cashflow.interest_rate.set_value(el) #Update en el interest rate del objeto
    
    else: #Caso en que estemos pasado del start date
        
        p1 = val_date.day_diff(ois_cashflow.start_date) 
        p2 = val_date.day_diff(ois_cashflow.end_date)
        
        fechas=Qcf.time_series_dates(fixings) #Obtener las fechas del fixing
        z=0
        P=1
        for i in fechas:
            tasa=Qcf.QCInterestRate(fixings[i], Qcf.QCAct360(), Qcf.QCLinearWf()) #Obtener las tasas de las fechas
            WF=tasa.wf(fechas[z],fechas[z+1]) #Calcular el WF
            P=P*WF #Multiplicar los WF
            z+=1 
            if i==val_date: #Si llegamos al val date, salir del ciclo
                break


        wf1 = P #WF
        wf2 = 1/(zcc.get_discount_factor_at(p2)) #WF de la zero cupón
        df_largo=zcc.get_discount_factor_at(p2) #Guardar el df para el present value
        el = (wf1*wf2 - 1) * 360.0 / (p2 - p1) #Calculo del el

        ois_cashflow.interest_rate.set_value(el) #Update en el interest rate del objeto
    
    pass
    # Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    # ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    # tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    # El ICP se puede pensar como el account asociado a la tasa interbancaria chilena,
    # que es el índice overnight chileno.
    #
    # Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.

In [20]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.

    """
    p2 = val_date.day_diff(ois_cashflow.end_date)
    df2 = zcc.get_discount_factor_at(p2)
     #Obtener el df largo 
    
    present=(ois_cashflow.amortization+ois_cashflow.interest_rate.get_value())*df2 #caculo del present value
    
    return present

# 

In [21]:
set_expected_rate(val_date,ois,zcc,fixings)
present_value(val_date,ois,zcc)

999268.2580778359

## Tests

In [22]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [23]:
amount_tol = 10000
rate_tol = .001

In [24]:
exitos = 0

In [25]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [26]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [27]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [28]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [29]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [30]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [31]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 0.70536602%


Se verificará el cálculo usando `df_fixings`.

In [32]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [33]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [34]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [35]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [36]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [37]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [38]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [39]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [40]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [41]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [42]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [43]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: -0.30454712%


In [44]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [45]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 50,551.23


In [46]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [47]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [48]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,052,488.94


In [49]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [50]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: -22538.106737


In [51]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [52]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [53]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [54]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [55]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [57]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.06121277%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [58]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.07438298%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [59]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.23464385%
Check tasa esperada es: 0.41387574%
Suma medio punto por obtener resultado.


In [60]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76596683%
Check tasa esperada es: 0.51628464%
Suma medio punto por obtener resultado.


In [61]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.76646371%
Check tasa esperada es: 0.51628464%
Suma medio punto por obtener resultado.


### `present_value`

In [63]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 999,116.5229
Amount: 1,052,488.9387
Df: 99.911652%
VP check es: 1,051,559.0882
Suma medio punto por obtener resultado.


In [64]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 999,296.5280
Amount: 1,052,488.9387
Df: 99.929653%
VP check es: 1,051,748.5414
Suma medio punto por obtener resultado.


In [65]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 999,350.1969
Amount: 1,052,488.9387
Df: 99.935019%
VP check es: 1,051,805.0256
Suma medio punto por obtener resultado.


In [66]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,000,000.0077
Amount: 1,052,488.9387
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [67]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Valor presente es: 1,000,031.1066
Amount: 1,052,488.9387
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [68]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 9.5
